In [1]:
from difflib import  unified_diff as diff
import sqlite3, json
from requests import Session
from xml.etree import ElementTree as XML
#conn = None
if 'conn' in dir():
  print('Previous Database Closed')
  conn.close()
conn = sqlite3.connect('database.db')
cur = conn.cursor()

The Features we'll be saving as follows:
- Title of the page
- comment of the edit
- revision ID of the current and previous edit
- difference of the content
- Username or IP

Because these can be hiiden by an admin.

In [2]:
SQL_INIT = """
CREATE TABLE IF NOT EXISTS `Revisions` (
  `RevID` INTEGER PRIMARY KEY,
  `Title` TEXT NOT NULL,
  `ParentID` INTEGER NULL DEFAULT NULL,
  `Comment` TEXT NULL DEFAULT NULL,
  `Anonymous` BOOLEAN DEFAULT FALSE,
  `Editor` VARCHAR(90) NULL DEFAULT NULL,
  `Text` TEXT NULL DEFAULT NULL,
  `Deleted` BOOLEAN DEFAILT FALSE
) WITHOUT ROWID;
CREATE INDEX IF NOT EXISTS `ParentID` ON `Revisions`(`ParentID`);
CREATE TABLE IF NOT EXISTS `Labels` (
  `RevID` INTEGER REFERENCES `Revisions` (`RevID`),
  `Labeller` VARCHAR(100) NOT NULL,
  `Vandalism` BOOLEAN DEFAULT FALSE,
  `Timestamp` TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
SQL_INSERT_REVISION = """
INSERT INTO `Revisions`
  (`Title` ,`RevID`, `ParentID`, `Comment`, `Anonymous`, `Editor`, `Deleted`, `Text`)
VALUES
  (?, ?,?,?,?,?,?, ?);
"""
SQL_UPDATE_REVISION = """
UPDATE `Revisions` SET `Text` = ? WHERE `ParentID` = ?
"""
cur.executescript(SQL_INIT)

In [3]:
# the parameter to pass on
mw = 'http://www.mediawiki.org/xml/export-0.11/' #The namespace
api_endpoint = 'https://bn.wikipedia.org/w/api.php' #Wikipedia API endpoint
ses = Session()
cache={}
insert_list = set()
backlog = set() # The Parent IDs which should be fetched
params_live = {
	"action": "query",
	"format": "json",
	"prop": "revisions",
	"utf8": 1,
  "indexpageids":1,
	"rvprop": "comment|content|ids|flags|user",
	"rvslots": "main"
}

In [4]:
def process_batch(query={}, ids=[], cache={}):
  """This function will be called on update statement each batch of results
  `queries`: The list of the values
  `cache`: A dictionary with `ParentID` as key and `Current revision` as Value
  """
  global backlog
  for id in ids:
    page = query[id]
    if 'revisions' not in page:
      print('Revision not found', page['title'])
      continue
    title = page['title']
    revision = page['revisions'][0]
    current_text = revision['slots']['main']['*'].splitlines()
    current_id = revision['revid']
    parent_id = revision['parentid']
    comment = revision['comment']
    editor = revision['user']
    insertable_text = None
    print(f'current Backlog size : {len(backlog)}')
    backlog.discard(current_id)
    print(f'current Backlog size : {len(backlog)}')
    if current_id in cache:
      print("""Put the difference""")
      insertable_text = '\n'.join(diff(cache[current_id], current_text))
      del cache[current_id]
      # if parent_id:
      #   # Add parent to the backlog
      #   backlog.add(parent_id)'
      #   print('Push to cache')
      #   cache[parent_id] = current_text
      print('Update parameters')
      yield insertable_text, current_id
    else:
      print(f'cache[{current_id}] not found', title)
      if not parent_id:
        print('First Revision', title)
        """
        ParentID does not exist
        So this is the first revision
        Insert it in raw format
        """
        parent_id = None
        insertable_text = '\n'.join(diff('', current_text))
      else:
        cache[parent_id] = current_text
        backlog.add(parent_id)
      #Insert into database
      insert_list.add(
          (
          title,
          current_id, parent_id,
          comment, 1, editor, 1,
          insertable_text)
      )

In [5]:
def fetch(ids = [], last_time = 0, grccontinue=None, rvcontinue=None, limit=100, idlimit=50):
  global cache, insert_list, backlog
  k=10
  while k:
    print('Epoch', k)
    if grccontinue:
      print('Generator Continuation')
      data = {**params_live,
        "generator": "recentchanges",
        "grcstart": last_time,
        "grcdir": "newer",
        "grclimit": limit,
        "grctoponly": 1,
        'grccontinue' : grccontinue
      }
    elif ids:
      ids = list(map(str, backlog.union(ids)))
      next_ids = []
      if len(ids) > idlimit:
        next_ids, ids = ids[idlimit:], ids[:idlimit]
      print('Current ID given', len(ids))
      data = {**params_live, 'revids':'|'.join(ids)}
      ids = next_ids
      print('Revision IDs are Given', data['revids'])
    elif last_time:
      print('Fetching Live Changes')
      data = {**params_live,
        "generator": "recentchanges",
        "grcstart": last_time,
        "grcdir": "newer",
        "grclimit": limit,
        "grctoponly": 1
      }
    else:
      print('neither Last Time nor Revision ID defined')
      
    if rvcontinue:
      # Revision continual
      data['rvcontinue'] = rvcontinue
    res = ses.post(api_endpoint, params=data, headers={'content-type':'application/json'}).json()
    
    if 'query' not in res:
      print('Query Not found')
      print(res)
      return
    cur.executemany(SQL_UPDATE_REVISION, process_batch(ids=res['query']['pageids'], query=res['query']['pages'], cache=cache))
    for i in insert_list:
      try:
        cur.execute(SQL_INSERT_REVISION, i)
      except sqlite3.IntegrityError as e:
        print(f'Error {i[0]}')
        pass
    insert_list = set()
    if 'continue' in res:
      print('Continual', res['continue'])
      grccontinue = res['continue']['grccontinue']
    if backlog:
      ids = backlog
    k -= 1
  
#cache={}
# fetch(ids=[123456, 123457]))
fetch(last_time='2021-08-28T07:34:00.000Z')

Epoch 10
Fetching Live Changes
current Backlog size : 0
current Backlog size : 0
cache[5369877] not found আরারিয়া জেলা
current Backlog size : 1
current Backlog size : 1
cache[5369869] not found আর্নল্ড শোয়ার্জনেগার
current Backlog size : 2
current Backlog size : 2
cache[5369781] not found আহমদ শাহ দুররানি
current Backlog size : 3
current Backlog size : 3
cache[5369784] not found আহমেদ শাহ বাহাদুর
current Backlog size : 4
current Backlog size : 4
cache[5369870] not found ইংল্যান্ড মহিলা ক্রিকেট দল
current Backlog size : 5
current Backlog size : 5
cache[5369872] not found ইনকুইজিশন
Revision not found ইয়োনাতান আদিদ নুনিয়েস
Revision not found উম্মে হানি বিনতে আবি তালিব
Revision not found কর্পোরেশন
current Backlog size : 6
current Backlog size : 6
cache[5369852] not found কাইমুর জেলা
current Backlog size : 7
current Backlog size : 7
cache[5369874] not found কিশানগঞ্জ জেলা
current Backlog size : 8
current Backlog size : 8
cache[5369760] not found কুচমান সিটি
Revision not found কোডারমা জে

In [6]:
#cur.execute('DELETE FROM Revisions')
for i in cur.execute('SELECT * FROM Revisions LIMIT 1'):
    print(i)
#print(cache)

(5369732, 'তিজারা', 5343820, '#WPWPBN #WPWP', 1, 'Hasan muntaseer', "--- \n\n+++ \n\n@@ -56,8 +56,6 @@\n\n | website                 = \n | footnotes               = \n }}\n-\n-[[File:Tijara.jpg|right|200px|thumb|তিজারা দুর্গ, এখন একটি হোটেল]]\n '''তিজারা''' ({{lang-en|Tijara}}) [[ভারত|ভারতের]] [[রাজস্থান]] রাজ্যের [[আলওয়ার জেলা|আলওয়ার]] জেলার একটি শহর ও পৌরসভা এলাকা।\n \n == ভৌগোলিক উপাত্ত ==", 1)


In [7]:
print(insert_list)

set()


In [8]:
cache

{5357686: ['{{তথ্যছক ভারতের জেলা',
  '| নাম                 = আরারিয়া',
  '| স্থানীয়_নাম        = [[File:Araria_Jila_in_Mithilakshar.png|100px]] <br /> اریا ضلع',
  '| রাজ্য               = বিহার',
  '| বিভাগ               = [[পূর্ণিয়া বিভাগ|পূর্ণিয়া]]',
  '| সদর                 = আরারিয়া',
  '| প্রতিষ্ঠিত          =',
  '| মানচিত্র            = Bihar district location map Araria.svg',
  '| স্থানাঙ্ক           =',
  '| আয়তন               = ২৮৩০',
  '| জনসংখ্যা            = ২,৮০৬,২০০',
  '| বছর                 = ২০১১',
  '| নগর                 =',
  '| সাক্ষরতা            = ৫৩.১%',
  '| লিঙ্গানুপাত         = ৯২১',
  '| সমাহর্তা            =',
  '| তহশিলসমূহ           =',
  '| লোকসভা              = [[আরারিয়া লোকসভা কেন্দ্র|আরারিয়া]]',
  '| বিধানসভা            = [[নরপতগঞ্জ বিধানসভা কেন্দ্র|নরপতগঞ্জ]], [[রানিগঞ্জ বিধানসভা কেন্দ্র, বিহার|রানিগঞ্জ]], [[Forbesganj ফোর্বসগঞ্জ বিধানসভা কেন্দ্র|ফোর্বসগঞ্জ]], [[আরারিয়া বিধানসভা কেন্দ্র|আরারিয়া]], [[জোকিহাট বিধানসভা কেন্দ্র|জোকিহাট]], [[